In [6]:
%pip install chromadb


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image, ImageFile
from pathlib import Path
from tqdm import tqdm
import json
import chromadb  # Заменили faiss на chromadb
from datetime import datetime

ImageFile.LOAD_TRUNCATED_IMAGES = True

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Device: {device}")


Device: mps


## 1. Загрузка датасета

In [8]:
# Загружаем CSV с информацией об изображениях
df = pd.read_csv('../data/balanced_animals_dataset.csv')
print(f"Всего изображений: {len(df)}")
print(f"\nКлассы: {df['scientific_name'].unique()}")
print(f"\nРаспределение по классам:\n{df['scientific_name'].value_counts()}")

Всего изображений: 12000

Классы: ['Canis aureus' 'Canis familiaris' 'Canis familiaris dingo'
 'Canis latrans' 'Canis lupus']

Распределение по классам:
scientific_name
Canis aureus              2400
Canis familiaris          2400
Canis familiaris dingo    2400
Canis latrans             2400
Canis lupus               2400
Name: count, dtype: int64


## 2. Создание экстрактора эмбеддингов

In [9]:
class EmbeddingExtractor(nn.Module):
    """Модель для извлечения эмбеддингов из различных архитектур"""
    
    def __init__(self, base_model, model_type='efficientnet'):
        super().__init__()
        self.model_type = model_type
        
        if model_type == 'efficientnet':
            # Для EfficientNet: features + avgpool
            self.features = base_model.features
            self.avgpool = base_model.avgpool
        elif model_type == 'resnet':
            # Для ResNet: все слои кроме fc
            self.conv1 = base_model.conv1
            self.bn1 = base_model.bn1
            self.relu = base_model.relu
            self.maxpool = base_model.maxpool
            self.layer1 = base_model.layer1
            self.layer2 = base_model.layer2
            self.layer3 = base_model.layer3
            self.layer4 = base_model.layer4
            self.avgpool = base_model.avgpool
        else:
            raise ValueError(f"Неподдерживаемый тип модели: {model_type}")
        
    def forward(self, x):
        if self.model_type == 'efficientnet':
            x = self.features(x)
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
        elif self.model_type == 'resnet':
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.relu(x)
            x = self.maxpool(x)
            
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)
            
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
        
        return x


def load_embedding_extractor(checkpoint_path):
    """Загружает модель и создает экстрактор эмбеддингов"""
    
    # Загружаем checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    params = checkpoint['params']
    
    print(f"Загружена модель: {params['model']['name']}")
    print(f"Количество классов: {params['model']['num_classes']}")
    
    # Создаем базовую модель
    model_name = params['model']['name']
    num_classes = params['model']['num_classes']
    
    if model_name == 'efficientnet_v2_m':
        weights = getattr(models.EfficientNet_V2_M_Weights, params['model']['pretrained'])
        base_model = models.efficientnet_v2_m(weights=weights)
        num_features = base_model.classifier[1].in_features
        base_model.classifier[1] = nn.Linear(num_features, num_classes)
        model_type = 'efficientnet'
        
    elif model_name == 'resnet50':
        weights = getattr(models.ResNet50_Weights, params['model']['pretrained'])
        base_model = models.resnet50(weights=weights)
        num_features = base_model.fc.in_features
        base_model.fc = nn.Linear(num_features, num_classes)
        model_type = 'resnet'
        
    else:
        raise ValueError(f"Модель {model_name} не поддерживается. Поддерживаются: efficientnet_v2_m, resnet50")
    
    # Загружаем веса
    base_model.load_state_dict(checkpoint['model_state_dict'])
    
    # Создаем экстрактор эмбеддингов (без классификационного слоя)
    embedding_extractor = EmbeddingExtractor(base_model, model_type=model_type)
    embedding_extractor.eval()
    embedding_extractor = embedding_extractor.to(device)
    
    # Определяем размерность эмбеддинга
    with torch.no_grad():
        dummy_input = torch.randn(1, 3, 224, 224).to(device)
        dummy_output = embedding_extractor(dummy_input)
        embedding_dim = dummy_output.shape[1]
    
    print(f"Размерность эмбеддинга: {embedding_dim}")
    print(f"Тип модели: {model_type}")
    
    return embedding_extractor, embedding_dim, checkpoint


In [10]:
# Загружаем модель
model_path = Path('../models/best_model.pth')
embedding_extractor, embedding_dim, checkpoint = load_embedding_extractor(model_path)

# Извлекаем маппинги классов
idx_to_label = checkpoint['idx_to_label']
label_to_idx = checkpoint['label_to_idx']

print(f"\nМаппинг классов:")
for idx, label in idx_to_label.items():
    print(f"  {idx}: {label}")

Загружена модель: resnet50
Количество классов: 5
Размерность эмбеддинга: 2048
Тип модели: resnet

Маппинг классов:
  0: Canis aureus
  1: Canis familiaris
  2: Canis familiaris dingo
  3: Canis latrans
  4: Canis lupus


## 3. Подготовка DataLoader

In [11]:
class ImageDataset(Dataset):
    """Dataset для загрузки изображений"""
    
    def __init__(self, dataframe, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform
        self.base_path = Path('../animal_images')
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        species = row['scientific_name'].replace(' ', '_')
        img_path = self.base_path / species / f"{row['uuid']}.jpg"
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Ошибка загрузки {img_path}: {e}")
            image = Image.new('RGB', (224, 224), color=(0, 0, 0))
        
        if self.transform:
            image = self.transform(image)
        
        # Возвращаем изображение и метаданные
        metadata = {
            'uuid': row['uuid'],
            'scientific_name': row['scientific_name'],
            'path': str(img_path.relative_to(Path('../'))),
            'index': idx
        }
        
        return image, metadata


# Трансформации (такие же как при валидации)
transform = transforms.Compose([
    transforms.Resize(384),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Создаем dataset и dataloader
dataset = ImageDataset(df, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)

print(f"Создан DataLoader с {len(dataset)} изображениями")

Создан DataLoader с 12000 изображениями


## 4. Извлечение эмбеддингов

In [12]:
def extract_embeddings(model, dataloader, device):
    """Извлекает эмбеддинги для всех изображений"""
    
    all_embeddings = []
    all_metadata = []
    
    model.eval()
    
    with torch.no_grad():
        for images, metadata_batch in tqdm(dataloader, desc="Извлечение эмбеддингов"):
            images = images.to(device)
            
            # Получаем эмбеддинги
            embeddings = model(images)
            
            embeddings_np = embeddings.cpu().numpy().astype(np.float32)
            embeddings_np = np.ascontiguousarray(embeddings_np)
            all_embeddings.append(embeddings_np)
            
            # Сохраняем метаданные
            for i in range(len(metadata_batch['uuid'])):
                meta = {
                    'uuid': metadata_batch['uuid'][i],
                    'scientific_name': metadata_batch['scientific_name'][i],
                    'path': metadata_batch['path'][i],
                    'embedding_index': len(all_metadata)
                }
                all_metadata.append(meta)
    
    # Объединяем все эмбеддинги
    all_embeddings = np.vstack(all_embeddings)
    all_embeddings = np.ascontiguousarray(all_embeddings, dtype=np.float32)
    
    print(f"\nИзвлечено эмбеддингов: {all_embeddings.shape}")
    print(f"Метаданных: {len(all_metadata)}")
    print(f"Тип данных: {all_embeddings.dtype}")
    print(f"Contiguous: {all_embeddings.flags['C_CONTIGUOUS']}")
    
    return all_embeddings, all_metadata


In [13]:
# Извлекаем эмбеддинги
embeddings, metadata = extract_embeddings(embedding_extractor, dataloader, device)

print(f"\nФорма массива эмбеддингов: {embeddings.shape}")
print(f"Тип данных: {embeddings.dtype}")
print(f"\nПример метаданных первого изображения:")
print(json.dumps(metadata[0], indent=2, ensure_ascii=False))

Извлечение эмбеддингов: 100%|██████████| 375/375 [01:22<00:00,  4.52it/s]


Извлечено эмбеддингов: (12000, 2048)
Метаданных: 12000
Тип данных: float32
Contiguous: True

Форма массива эмбеддингов: (12000, 2048)
Тип данных: float32

Пример метаданных первого изображения:
{
  "uuid": "710acdf3-5470-44ca-a1ac-32fb42c95b70",
  "scientific_name": "Canis aureus",
  "path": "animal_images/Canis_aureus/710acdf3-5470-44ca-a1ac-32fb42c95b70.jpg",
  "embedding_index": 0
}


## 5. Построение chroma индекса

In [14]:
def build_chromadb_collection(embeddings, metadata, collection_name="animal_embeddings"):
    """Строит ChromaDB коллекцию для поиска похожих векторов
    
    Args:
        embeddings: numpy array с эмбеддингами (n_samples, embedding_dim)
        metadata: список словарей с метаданными для каждого изображения
        collection_name: название коллекции
    
    Returns:
        client: ChromaDB клиент
        collection: ChromaDB коллекция
    """
    
    print(f"Создание ChromaDB коллекции '{collection_name}'...")
    
    # Создаем persistent клиент (сохраняется на диск)
    chroma_path = Path('../embeddings/chromadb')
    chroma_path.mkdir(parents=True, exist_ok=True)
    
    client = chromadb.PersistentClient(path=str(chroma_path))
    
    # Удаляем коллекцию если существует (для чистого старта)
    try:
        client.delete_collection(name=collection_name)
        print("Старая коллекция удалена")
    except:
        pass
    
    # Создаем новую коллекцию с косинусным расстоянием
    collection = client.create_collection(
        name=collection_name,
        metadata={"hnsw:space": "cosine"}  # cosine, l2, или ip
    )
    
    # Подготавливаем данные для ChromaDB
    ids = [meta['uuid'] for meta in metadata]
    embeddings_list = embeddings.tolist()
    
    # Подготавливаем метаданные (ChromaDB требует строковые значения)
    metadatas = [
        {
            'scientific_name': meta['scientific_name'],
            'path': meta['path'],
            'embedding_index': str(meta['embedding_index'])
        }
        for meta in metadata
    ]
    
    # Добавляем данные батчами (ChromaDB рекомендует батчи ~40000)
    batch_size = 1000
    num_batches = (len(ids) + batch_size - 1) // batch_size
    
    print(f"Добавление {len(ids)} векторов в {num_batches} батчах...")
    
    for i in tqdm(range(0, len(ids), batch_size), desc="Загрузка в ChromaDB"):
        batch_end = min(i + batch_size, len(ids))
        
        collection.add(
            ids=ids[i:batch_end],
            embeddings=embeddings_list[i:batch_end],
            metadatas=metadatas[i:batch_end]
        )
    
    print(f"✓ Коллекция создана. Количество векторов: {collection.count()}")
    
    return client, collection


In [15]:
# Строим ChromaDB коллекцию
client, collection = build_chromadb_collection(embeddings, metadata)

Создание ChromaDB коллекции 'animal_embeddings'...
Старая коллекция удалена
Добавление 12000 векторов в 12 батчах...


Загрузка в ChromaDB: 100%|██████████| 12/12 [00:07<00:00,  1.63it/s]

✓ Коллекция создана. Количество векторов: 12000


## 6. Тестирование поиска

In [16]:
def test_search(collection, metadata, test_idx=0, k=10):
    """Тестирует поиск похожих изображений в ChromaDB"""
    
    # Получаем UUID тестового изображения
    query_uuid = metadata[test_idx]['uuid']
    
    print(f"\nТестовое изображение (индекс {test_idx}):")
    print(f"  UUID: {metadata[test_idx]['uuid']}")
    print(f"  Класс: {metadata[test_idx]['scientific_name']}")
    print(f"  Путь: {metadata[test_idx]['path']}")
    
    # Получаем эмбеддинг из коллекции
    query_result = collection.get(
        ids=[query_uuid],
        include=['embeddings']
    )
    
    query_embedding = query_result['embeddings'][0]
    
    # Ищем k ближайших соседей
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k,
        include=['metadatas', 'distances']
    )
    
    print(f"\nТоп-{k} похожих изображений:")
    for i, (uuid, distance, meta) in enumerate(zip(
        results['ids'][0], 
        results['distances'][0], 
        results['metadatas'][0]
    )):
        # Для косинусного расстояния: similarity = 1 - distance
        similarity = (1 - distance) * 100
        print(f"  {i+1}. Похожесть: {similarity:.2f}% | Класс: {meta['scientific_name']} | UUID: {uuid}")
    
    return results


In [17]:
# Тестируем поиск на нескольких примерах
print("=" * 80)
print("ТЕСТ 1: Первое изображение")
print("=" * 80)
test_search(collection, metadata, test_idx=0, k=10)

print("\n" + "=" * 80)
print("ТЕСТ 2: Случайное изображение")
print("=" * 80)
random_idx = np.random.randint(0, len(metadata))
test_search(collection, metadata, test_idx=random_idx, k=10)


ТЕСТ 1: Первое изображение

Тестовое изображение (индекс 0):
  UUID: 710acdf3-5470-44ca-a1ac-32fb42c95b70
  Класс: Canis aureus
  Путь: animal_images/Canis_aureus/710acdf3-5470-44ca-a1ac-32fb42c95b70.jpg

Топ-10 похожих изображений:
  1. Похожесть: 100.00% | Класс: Canis aureus | UUID: 710acdf3-5470-44ca-a1ac-32fb42c95b70
  2. Похожесть: 91.60% | Класс: Canis aureus | UUID: 06a9b4ca-e660-4bd4-906b-c88f88bd52b4
  3. Похожесть: 89.23% | Класс: Canis aureus | UUID: e7e91bf5-7605-4385-bca1-b4853091b915
  4. Похожесть: 88.40% | Класс: Canis aureus | UUID: f1d1451d-0cc9-4ce5-ab5a-be44112b5cfa
  5. Похожесть: 88.38% | Класс: Canis aureus | UUID: 8db57231-ade9-44f3-85d3-ae9da509092c
  6. Похожесть: 88.31% | Класс: Canis aureus | UUID: 2bc7c25f-8931-4f48-a534-0ad99b39aaee
  7. Похожесть: 88.30% | Класс: Canis aureus | UUID: 63bebc7b-c974-44d4-8cb4-2e34fc60eb3f
  8. Похожесть: 87.84% | Класс: Canis aureus | UUID: 8874bd71-ff6c-4bd2-b9de-04a27426b359
  9. Похожесть: 87.63% | Класс: Canis aureus |

{'ids': [['f4e5afce-4d24-4a35-b696-4ef5322cb87f',
   '03a5c1d1-98ea-44c4-a7ba-f6c6a5994fc6',
   'd128f8b5-9d9d-46e1-8e3b-b8efe4c8a409',
   'ffc1a61f-f48f-4ecb-b634-a815a1440686',
   '43891b83-3938-4a6d-b7b2-e6ef2b86ae15',
   '39fadfde-dc8b-4d2d-98f2-e7955cfa6f21',
   '5896eb2e-9daa-47ae-a0f6-a5fc19d0d0c0',
   '9176cab6-24fc-4db0-a3b9-3ffcfb32a369',
   '296e7261-a557-4041-ab75-285b580a689a',
   '2a315951-5ad8-4f84-8258-b96aae779782']],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'included': ['metadatas', 'distances'],
 'data': None,
 'metadatas': [[{'embedding_index': '1583',
    'scientific_name': 'Canis aureus',
    'path': 'animal_images/Canis_aureus/f4e5afce-4d24-4a35-b696-4ef5322cb87f.jpg'},
   {'scientific_name': 'Canis aureus',
    'embedding_index': '2266',
    'path': 'animal_images/Canis_aureus/03a5c1d1-98ea-44c4-a7ba-f6c6a5994fc6.jpg'},
   {'embedding_index': '6033',
    'path': 'animal_images/Canis_familiaris_dingo/d128f8b5-9d9d-46e1-8e3b-b8efe4c8a409.jpg',
    

## 7. Сохранение индекса и метаданных

In [18]:
# Создаем директорию для сохранения
embeddings_dir = Path('../embeddings')
embeddings_dir.mkdir(exist_ok=True)

# ChromaDB уже сохранен в persistent mode
print(f"✓ ChromaDB сохранен в: {embeddings_dir / 'chromadb'}")

# Сохраняем метаданные
metadata_dict = {
    'images': metadata,
    'metadata': {
        'total_images': len(metadata),
        'embedding_dim': embedding_dim,
        'model': checkpoint['params']['model']['name'],
        'use_cosine_similarity': True,
        'created_at': datetime.now().isoformat(),
        'classes': list(idx_to_label.values()),
        'collection_name': 'animal_embeddings'
    }
}

metadata_path = embeddings_dir / 'image_metadata.json'
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata_dict, f, indent=2, ensure_ascii=False)
print(f"✓ Метаданные сохранены: {metadata_path}")

# Опционально: сохраняем сами эмбеддинги (для анализа)
embeddings_path = embeddings_dir / 'embeddings.npy'
np.save(embeddings_path, embeddings)
print(f"✓ Эмбеддинги сохранены: {embeddings_path}")

# Статистика размеров
import shutil
chroma_size = sum(f.stat().st_size for f in (embeddings_dir / 'chromadb').rglob('*') if f.is_file())
print(f"\nВсе файлы сохранены в директории: {embeddings_dir}")
print(f"Размер ChromaDB: {chroma_size / 1024 / 1024:.2f} MB")
print(f"Размер метаданных: {metadata_path.stat().st_size / 1024:.2f} KB")
print(f"Размер эмбеддингов: {embeddings_path.stat().st_size / 1024 / 1024:.2f} MB")


✓ ChromaDB сохранен в: ../embeddings/chromadb
✓ Метаданные сохранены: ../embeddings/image_metadata.json
✓ Эмбеддинги сохранены: ../embeddings/embeddings.npy

Все файлы сохранены в директории: ../embeddings
Размер ChromaDB: 173.46 MB
Размер метаданных: 2680.16 KB
Размер эмбеддингов: 93.75 MB


## 8. Проверка загрузки

In [19]:
# Проверяем, что ChromaDB можно загрузить обратно
print("Проверка загрузки ChromaDB...")

# Загружаем существующий клиент
loaded_client = chromadb.PersistentClient(path=str(embeddings_dir / 'chromadb'))
loaded_collection = loaded_client.get_collection(name="animal_embeddings")
print(f"✓ ChromaDB загружен. Количество векторов: {loaded_collection.count()}")

with open(metadata_path, 'r', encoding='utf-8') as f:
    loaded_metadata = json.load(f)
print(f"✓ Метаданные загружены. Количество изображений: {loaded_metadata['metadata']['total_images']}")

loaded_embeddings = np.load(embeddings_path)
print(f"✓ Эмбеддинги загружены. Форма: {loaded_embeddings.shape}")

print("\n✅ Все файлы успешно сохранены и загружены!")

# Тестовый поиск на загруженной коллекции
print("\n" + "=" * 80)
print("ТЕСТ: Поиск в загруженной коллекции")
print("=" * 80)
test_idx = 100
test_search(loaded_collection, loaded_metadata['images'], test_idx=test_idx, k=5)


Проверка загрузки ChromaDB...
✓ ChromaDB загружен. Количество векторов: 12000
✓ Метаданные загружены. Количество изображений: 12000
✓ Эмбеддинги загружены. Форма: (12000, 2048)

✅ Все файлы успешно сохранены и загружены!

ТЕСТ: Поиск в загруженной коллекции

Тестовое изображение (индекс 100):
  UUID: e14cf0ed-919f-4d09-8f9a-eff023047033
  Класс: Canis aureus
  Путь: animal_images/Canis_aureus/e14cf0ed-919f-4d09-8f9a-eff023047033.jpg

Топ-5 похожих изображений:
  1. Похожесть: 100.00% | Класс: Canis aureus | UUID: e14cf0ed-919f-4d09-8f9a-eff023047033
  2. Похожесть: 78.75% | Класс: Canis lupus | UUID: 7661b946-70c0-4322-a0ae-74b7970785ac
  3. Похожесть: 78.72% | Класс: Canis latrans | UUID: 9d367e2a-9dfe-4e70-88f3-04b66d49170a
  4. Похожесть: 78.29% | Класс: Canis lupus | UUID: be2d4b4c-6fae-4b6c-a505-803e3e694701
  5. Похожесть: 78.07% | Класс: Canis latrans | UUID: 0a864fe7-868c-483f-97a9-fe7ad7a0c39b


{'ids': [['e14cf0ed-919f-4d09-8f9a-eff023047033',
   '7661b946-70c0-4322-a0ae-74b7970785ac',
   '9d367e2a-9dfe-4e70-88f3-04b66d49170a',
   'be2d4b4c-6fae-4b6c-a505-803e3e694701',
   '0a864fe7-868c-483f-97a9-fe7ad7a0c39b']],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'included': ['metadatas', 'distances'],
 'data': None,
 'metadatas': [[{'embedding_index': '100',
    'scientific_name': 'Canis aureus',
    'path': 'animal_images/Canis_aureus/e14cf0ed-919f-4d09-8f9a-eff023047033.jpg'},
   {'path': 'animal_images/Canis_lupus/7661b946-70c0-4322-a0ae-74b7970785ac.jpg',
    'scientific_name': 'Canis lupus',
    'embedding_index': '9731'},
   {'embedding_index': '8928',
    'scientific_name': 'Canis latrans',
    'path': 'animal_images/Canis_latrans/9d367e2a-9dfe-4e70-88f3-04b66d49170a.jpg'},
   {'embedding_index': '11334',
    'path': 'animal_images/Canis_lupus/be2d4b4c-6fae-4b6c-a505-803e3e694701.jpg',
    'scientific_name': 'Canis lupus'},
   {'path': 'animal_images/Canis_latr

## 9. Статистика и визуализация

In [20]:
# Статистика по классам
class_counts = {}
for meta in metadata:
    class_name = meta['scientific_name']
    class_counts[class_name] = class_counts.get(class_name, 0) + 1

print("Распределение изображений по классам:")
for class_name, count in sorted(class_counts.items()):
    print(f"  {class_name}: {count} изображений")

print(f"\nВсего классов: {len(class_counts)}")
print(f"Всего изображений: {sum(class_counts.values())}")

Распределение изображений по классам:
  Canis aureus: 2400 изображений
  Canis familiaris: 2400 изображений
  Canis familiaris dingo: 2400 изображений
  Canis latrans: 2400 изображений
  Canis lupus: 2400 изображений

Всего классов: 5
Всего изображений: 12000
